<a href="https://colab.research.google.com/github/Swathi1309/Medical-Image-Analysis/blob/main/Locating_the_optic_nerve_in_retinal_image.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Locating the optic nerve in a retinal image using fuzzy convergence of blood vessels

## Importing libraries

In [ ]:
import numpy as np
import cv2
from google.colab.patches import cv2_imshow

from google.colab import drive
drive.mount('/content/drive')

import math
from scipy import signal

import matplotlib.pyplot as plt

from skimage.morphology import thin

import sys
sys.setrecursionlimit(3000)

import timeit
import copy as cp
import os

## Supporting functions - segmentation

In [54]:
# Function to check if index is inside image
def inbounds(shape, indices):
    for i, ind in enumerate(indices):
        if ind < 0 or ind >= shape[i]:
            return False
    return True

# Function to break image at all junctions (where two or more segments intersect)
def break_image(image):
  broken = np.zeros(image.shape)
  for i in range(image.shape[0]):
    for j in range(image.shape[1]):
      if get_neighbourhood(image, i, j)>2:
        broken[i,j]=0
      else:
        broken[i,j]=image[i,j]
  return broken
  
# Function to get number of white pixels in the 8 neighbourhood of an image
def get_neighbourhood(image, i, j):
  total = 0
  if inbounds(image.shape, (i-1, j)):
    total+=image[i-1, j]
  if inbounds(image.shape, (i, j-1)):
    total+=image[i, j-1]
  if inbounds(image.shape, (i+1, j)):
    total+=image[i+1, j]
  if inbounds(image.shape, (i, j+1)):
    total+=image[i, j+1]
  if inbounds(image.shape, (i-1, j-1)):
    total+=image[i-1, j-1]
  if inbounds(image.shape, (i+1, j-1)):
    total+=image[i+1, j-1]
  if inbounds(image.shape, (i+1, j+1)):
    total+=image[i+1, j+1]
  if inbounds(image.shape, (i-1, j+1)):
    total+=image[i-1, j+1]
  return total

# Function to erase all pixels below a given threshold in an image
def threshold(image, t):
  new_image = np.zeros(image.shape)
  for i in range(image.shape[0]):
    for j in range(image.shape[1]):
      if image[i,j]<t:
        new_image[i,j]=0
      else:
        new_image[i,j] =image[i,j]
  return new_image

# Set the label of a vessel
def setlable(img, labimg, x, y, label):
    if img[y,x] and not labimg[y,x]:
        labimg[y,x] = label
        if inbounds(img.shape, (y, x+1)):
            setlable(img, labimg, x+1, y,label)
        if inbounds(img.shape, (y+1, x)):
            setlable(img, labimg, x, y+1,label)
        if inbounds(img.shape, (y, x-1)):
            setlable(img, labimg, x-1, y,label)
        if inbounds(img.shape, (y-1, x)):
            setlable(img, labimg, x, y-1,label)
        if inbounds(img.shape, (y+1, x+1)):
            setlable(img, labimg, x+1, y+1,label)
        if inbounds(img.shape, (y+1, x-1)):
            setlable(img, labimg, x-1, y+1,label)
        if inbounds(img.shape, (y-1, x+1)):
            setlable(img, labimg, x+1, y-1,label)
        if inbounds(img.shape, (y-1, x-1)):
            setlable(img, labimg, x-1, y-1,label)

# Function to label the vessel
def labelvessel(img, labimg, point, thresh, size, listcd):
    if img[point[1], point[0]] >= thresh and not labimg[point[1], point[0]] and thresh:
        labimg[point[1], point[0]] = 1
        x = point[0]
        y = point[1]
        listcd.append([x, y])
        size += 1
        try:
            if size > 500:
                return False
            if inbounds(img.shape, (y, x+1)):
                labelvessel(img, labimg, (x+1, y),thresh, size, listcd)
            if inbounds(img.shape, (y+1, x)):
                labelvessel(img, labimg, (x, y+1),thresh, size, listcd)
            if inbounds(img.shape, (y, x-1)):
                labelvessel(img, labimg, (x-1, y),thresh, size, listcd)
            if inbounds(img.shape, (y-1, x)):
                labelvessel(img, labimg, (x, y-1),thresh, size, listcd)
            if inbounds(img.shape, (y+1, x+1)):
                labelvessel(img, labimg, (x+1, y+1),thresh, size, listcd)
            if inbounds(img.shape, (y+1, x-1)):
                labelvessel(img, labimg, (x-1, y+1),thresh, size, listcd)
            if inbounds(img.shape, (y-1, x-1)):
                labelvessel(img, labimg, (x-1, y-1),thresh, size, listcd)
            if inbounds(img.shape, (y-1, x+1)):
                labelvessel(img, labimg, (x+1, y-1),thresh, size, listcd)
        except Exception:
            print ("Error:", Exception, "in paint_fill...")

# Function to get all segments in an image
def get_segments(image, t1, t2):
  new_image = image.copy()
  visited = np.zeros(image.shape)
  segment_ends = np.zeros(image.shape)
  for i in range(new_image.shape[0]):
    for j in range(new_image.shape[1]):
      if new_image[i,j]==1 and visited[i,j]==0 and (get_neighbourhood(new_image, i, j)==1 or get_neighbourhood(new_image, i, j)==0):
        size = find_start_end(new_image, i, j, visited, segment_ends, 0)
        if size<=t1 or size>=t2:
          erase_segment(new_image, i, j, 0, size)
  return new_image, segment_ends

# Function to find start and end point of a segment
def find_start_end(image, i, j, visited, segment_ends, l):
  visited[i,j]=1
  l+=1
  if i-1>=0:
    if image[i-1, j]==1 and visited[i-1,j]==0:
      l = find_start_end(image, i-1, j, visited, segment_ends, l)
  if j-1>=0:
    if image[i, j-1]==1 and visited[i, j-1]==0:
      l = find_start_end(image, i, j-1, visited, segment_ends, l)
  if i+1<image.shape[0]:
    if image[i+1, j]==1 and visited[i+1, j]==0:
      l = find_start_end(image, i+1, j, visited, segment_ends, l)
  if j+1<image.shape[1]:
    if image[i, j+1]==1 and visited[i, j+1]==0:
      l = find_start_end(image, i, j+1, visited, segment_ends, l)
  if i-1>=0 and j-1>=0:
    if image[i-1, j-1]==1 and visited[i-1, j-1]==0:
      l = find_start_end(image, i-1, j-1, visited, segment_ends, l)
  if i+1<image.shape[0] and j-1>=0:
    if image[i+1, j-1]==1 and visited[i+1,j-1]==0:
      l = find_start_end(image, i+1, j-1, visited, segment_ends, l)
  if i+1<image.shape[0] and j+1<image.shape[1]:
    if image[i+1, j+1]==1 and visited[i+1, j+1]==0:
      l = find_start_end(image, i+1, j+1, visited, segment_ends, l)
  if i-1>=0 and j+1<image.shape[1]:
    if image[i-1, j+1]==1 and visited[i-1, j+1]==0:
      l = find_start_end(image, i-1, j+1, visited, segment_ends, l)
  if get_neighbourhood(image, i, j)==1 or get_neighbourhood(image, i, j)==0:
    segment_ends[i,j]=1
  return l

# Function to remove a segment
def erase_segment(image, i, j, l, size):
  l+=1
  image[i, j]=0
  if i-1>=0:
    if image[i-1, j]==1:
      erase_segment(image, i-1, j, l, size)
  if j-1>=0:
    if image[i, j-1]==1:
      erase_segment(image, i, j-1, l, size)
  if i+1<image.shape[0]:
    if image[i+1, j]==1:
      erase_segment(image, i+1, j, l, size)
  if j+1<image.shape[1]:
    if image[i, j+1]==1:
      erase_segment(image, i, j+1, l, size)
  if i-1>=0 and j-1>=0:
    if image[i-1, j-1]==1:
      erase_segment(image, i-1, j-1, l, size)
  if i+1<image.shape[0] and j-1>=0:
    if image[i+1, j-1]==1:
      erase_segment(image, i+1, j-1, l, size)
  if i+1<image.shape[0] and j+1<image.shape[1]:
    if image[i+1, j+1]==1:
      erase_segment(image, i+1, j+1, l, size)
  if i-1>=0 and j+1<image.shape[1]:
    if image[i-1, j+1]==1:
      erase_segment(image, i-1, j+1, l, size)

# Class with methods to implement threshold probing
class Probe:
    def __init__(self, thresh, smin, smax, fringe, tree):
        self.th = thresh
        self.smin = smin
        self.smax = smax
        self.fg = fringe
        self.tree = tree

    def init_queue(self, mfr0):
        originalimg = cp.copy(mfr0)
        mfr = cp.copy(mfr0)
        h, w = mfr.shape
        hist,bins = np.histogram(mfr.ravel(),256,[0,256])
        for i in range(len(hist)):
            if hist[i] > self.th:
                hist[i] = 0
        h, w = mfr.shape
        for y in range(h):
            for x in range(w):
                if not hist[mfr[y,x]]:
                    mfr[y,x] = 0
                else:
                    mfr[y,x] = 1
        threshimg = cp.copy(mfr)

        # Thin image
        thmfr = thin(mfr)

        # erase branchpoints
        for y in range(1,h-1,1):
            for x in range(1,w-1,1):
                if x == 0 or y == 0 or x == w-1 or y == h-1:
                    continue
                p2 = int(thmfr[y-1, x])
                p3 = int(thmfr[y-1, x+1])
                p4 = int(thmfr[y, x+1])
                p5 = int(thmfr[y+1, x+1])
                p6 = int(thmfr[y+1, x])
                p7 = int(thmfr[y+1, x-1])
                p8 = int(thmfr[y, x-1])
                p9 = int(thmfr[y-1,x-1])
                num = p2 + p3 + p4 + p5 + p6 + p7 + p8 + p9
                if num >= 3:
                    thmfr[y, x] = 0

        nonbranch = cp.copy(thmfr)

        # discard segments < 10 pixels
        lab = 1
        label = np.zeros(thmfr.shape)
        for y in range(h):
            for x in range(w):
                if not label[y, x] and thmfr[y, x]:
                    setlable(thmfr, label, x, y, lab)
                    lab += 1
        num = np.zeros(lab)
        for y in range(h):
            for x in range(w):
                num[int(label[y,x]-1)] += 1
        for y in range(h):
            for x in range(w):
                if num[int(label[y,x]-1)] <= 20 or num[int(label[y,x]-1)] >=100:
                    thmfr[y,x] = 0
        remove = cp.copy(thmfr)

        # find endpoints for queue
        queue = []

        for y in range(1,h-1,1):
            for x in range(1,w-1,1):
                if x == 0 or y == 0 or x == w-1 or y == h-1:
                    continue
                p2 = int(thmfr[y-1, x])
                p3 = int(thmfr[y-1, x+1])
                p4 = int(thmfr[y, x+1])
                p5 = int(thmfr[y+1, x+1])
                p6 = int(thmfr[y+1, x])
                p7 = int(thmfr[y+1, x-1])
                p8 = int(thmfr[y, x-1])
                p9 = int(thmfr[y-1,x-1])
                num = p2 + p3 + p4 + p5 + p6 + p7 + p8 + p9
                if num == 1:
                    queue.append([x, y])
        pointimg = cp.copy(thmfr)
        return queue

    def paint_fill(self, img, labelimg, p, T):
        size = 0
        listcd = []
        labelvessel(img, labelimg, p, T, size, listcd)
        return (np.count_nonzero(labelimg), labelimg, listcd)

    def tests(self, size, piece, T, vessel, listcd):
        if size > 30:

            # first, the size must less than smax
            if size > self.smax:
                return False

            # second, the threshold must be positive
            if T <= 1:
                return False

            # third, the piece cannot touch the vessel-classied pixel
            logpiece = piece > 0
            logvessel = vessel > 0
            result = logpiece & logvessel
            if result.sum() > 0:
                return False

            # fourth, border-pixels-touching-another-piece / total-pixel-in-piece
            h, w = piece.shape[:2]
            border = 0
            for x, y in listcd:
                if x == 0 or y == 0 or x == w-1 or y == h-1:
                    continue
                p2 = int(piece[y-1, x])
                p3 = int(piece[y-1, x+1])
                p4 = int(piece[y, x+1])
                p5 = int(piece[y+1, x+1])
                p6 = int(piece[y+1, x])
                p7 = int(piece[y+1, x-1])
                p8 = int(piece[y, x-1])
                p9 = int(piece[y-1,x-1])
                num = p2 + p3 + p4 + p5 + p6 + p7 + p8 + p9
                vp2 = int(vessel[y-1, x])
                vp3 = int(vessel[y-1, x+1])
                vp4 = int(vessel[y, x+1])
                vp5 = int(vessel[y+1, x+1])
                vp6 = int(vessel[y+1, x])
                vp7 = int(vessel[y+1, x-1])
                vp8 = int(vessel[y, x-1])
                vp9 = int(vessel[y-1,x-1])
                touch = vp2 + vp3 + vp4 + vp5 + vp6 + vp7 + vp8 + vp9
                if num != 8 and touch:
                    border += 1
            if (border / logpiece.sum()) > self.fg:
                return False

            # fifth, total-pixel-in-piece / branches-in-piece
            listcd.sort()
            temppiece, indexskeleton = indirectindexing(listcd, piece)
            branch = 0
            for x, y in indexskeleton:
                if x == 0 or y == 0 or x == w-1 or y == h-1:
                    continue
                p2 = int(temppiece[y-1, x])
                p3 = int(temppiece[y-1, x+1])
                p4 = int(temppiece[y, x+1])
                p5 = int(temppiece[y+1, x+1])
                p6 = int(temppiece[y+1, x])
                p7 = int(temppiece[y+1, x-1])
                p8 = int(temppiece[y, x-1])
                p9 = int(temppiece[y-1,x-1])
                num = p2 + p3 + p4 + p5 + p6 + p7 + p8 + p9
                if num >= 3:
                    branch += 1
            if (logpiece.sum() / branch) < self.tree:
                return False
            return True

        else:

            # second, the threshold must be positive
            if T <= 1:
                return False

            # third, the piece cannot touch the vessel-classied pixel
            logpiece = piece > 0
            logvessel = vessel > 0
            result = logpiece & logvessel
            if result.sum() > 0:
                return False
            return True

    def label(self, vessel, tempvessel):
        return (vessel | tempvessel)

    def addpoints(self, queue, vesselpiece, vessel, listcd):
        tempvessel, indexskeleton = indirectindexing(listcd, vesselpiece)
        h, w = piece.shape[:2]
        for x, y in indexskeleton:
            if x == 0 or y == 0 or x == w-1 or y == h-1:
                    continue
            p2 = int(tempvessel[y-1, x])
            p3 = int(tempvessel[y-1, x+1])
            p4 = int(tempvessel[y, x+1])
            p5 = int(tempvessel[y+1, x+1])
            p6 = int(tempvessel[y+1, x])
            p7 = int(tempvessel[y+1, x-1])
            p8 = int(tempvessel[y, x-1])
            p9 = int(tempvessel[y-1,x-1])
            num = p2 + p3 + p4 + p5 + p6 + p7 + p8 + p9
            if num == 1:
                point = [x, y]
                if not checkidentical(queue, point, vessel):
                    queue.append(point)
        return queue

    def deletepoint(self, queue, vessel, num):
        que = cp.copy(queue)
        count = 0
        for j in range(num, len(queue), 1):
            p = [0, 0]
            p[1], p[0] = queue[j][1], queue[j][0]
            num = vessel[p[1]][p[0]] + vessel[p[1]+1][p[0]] + vessel[p[1]][p[0]+1] +\
                  vessel[p[1]][p[0]-1] + vessel[p[1]-1][p[0]]
            if num > 2:
                que.pop(j-count)
                count += 1
        return que

# Function to check if two vessels are identical
def checkidentical(l, point, vessel):
    for i in l:
        if i == point:
            return True
    return False

# Function for indirect indexing
def indirectindexing(listcd, img):
    prev = np.zeros_like(img)
    diff = np.ones_like(img)
    indexskeleton = []
    while cv2.countNonZero(diff) > 15:
        img, indexskeleton = indirectIteration(listcd, img, indexskeleton, 0)
        img, indexskeleton = indirectIteration(listcd, img, indexskeleton, 1)
        diff = cv2.absdiff(img, prev)
        prev = cp.copy(img)
    return img, indexskeleton

# Function for indirect iteration
def indirectIteration(listcd, im, indexskeleton, iter):
    h, w = im.shape[:2]
    marker = np.ones(im.shape)
    for x, y in listcd:
        if x == 0 or y == 0 or x == w-1 or y == h-1:
            continue
        p2 = int(im[y-1, x])
        p3 = int(im[y-1, x+1])
        p4 = int(im[y, x+1])
        p5 = int(im[y+1, x+1])
        p6 = int(im[y+1, x])
        p7 = int(im[y+1, x-1])
        p8 = int(im[y, x-1])
        p9 = int(im[y-1,x-1])

        A = (p2 == 0 and p3 == 1) + (p3 == 0 and p4 == 1) + \
            (p4 == 0 and p5 == 1) + (p5 == 0 and p6 == 1) + \
            (p6 == 0 and p7 == 1) + (p7 == 0 and p8 == 1) + \
            (p8 == 0 and p9 == 1) + (p9 == 0 and p2 == 1)
        B = p2 + p3 + p4 + p5 + p6 + p7 + p8 + p9
        if iter == 0:
            m1 = p2 * p4 * p6
            m2 = p4 * p6 * p8
        else:
            m1 = p2 * p4 * p8
            m2 = p2 * p6 * p8
        if A == 1 and (B >= 2 and B <= 6) and m1 == 0 and m2 == 0:
            marker[y,x] = 0
    for x, y in listcd:
        if im[y, x] and marker[y, x]:
            im[y, x] = 1
            indexskeleton.append([x, y])
        else:
            im[y, x] = 0
    return im, indexskeleton

# Function to determine if two pieces touch each other
def touchpieces(vessel, temp):
    piece = cp.copy(temp)
    vessel = vessel > 0
    count = 0
    for i in range(len(temp)):
        piece[i] = piece[i] > 0
        touch = vessel & piece[i]
        if np.count_nonzero(touch):
            count += 1
    if count == 2 and len(temp):
        return True
    else:
        return False

# Function to produce a gaussian kernel
def kernel_gaussian(s, L, u, v):
  if abs(u)<=3*s and abs(v)<=L/2:
    return -1*math.exp(-1*(u*u)/(2*s*s))
  else:
    return 0

# MFR Convolution
def convolution_segmentation(image):
  kernels = []
  s = 1.5
  L = 9
  k = np.zeros((16,16))
  n = 0
  mean = 0
  for i in range(k.shape[0]):
    for j in range(k.shape[1]):
      k[i,j]=kernel_gaussian(s, L, i-7.5, j-7.5)
      if k[i,j]!=0:
        mean+=k[i,j]
        n+=1
  mean/=n
  for i in range(k.shape[0]):
    for j in range(k.shape[1]):
      if k[i,j]!=0:
        k[i,j]-=mean

  kernels.append(k)

  height, width = k.shape[:2]
  center = ((width-1)/2, (height-1)/2)

  for i in range(1,12):
    rotate_matrix = cv2.getRotationMatrix2D(center=center, angle=i*15, scale=1)
    new_image = cv2.warpAffine(src=k, M=rotate_matrix, dsize=(width, height))
    kernels.append(new_image)

  conv_image = []
  for i in range(12):
    img = cv2.filter2D(image, -1, kernels[i])
    conv_image.append(img)
  conv_image = np.max(conv_image, axis=0)
  mean = np.mean(conv_image)
  conv_image = threshold(conv_image, np.floor(3*mean))
  return conv_image

# Function to find the second edge of a segment given one edge
def get_edge(image, i, j):
  visited = np.zeros(image.shape)
  segment_ends = np.zeros(image.shape)
  visited[i,j]=1
  l=1
  if i-1>=0:
    if image[i-1, j]==1 and visited[i-1,j]==0:
      find_start_end(image, i-1, j, visited, segment_ends, l)
  if j-1>=0:
    if image[i, j-1]==1 and visited[i, j-1]==0:
      find_start_end(image, i, j-1, visited, segment_ends, l)
  if i+1<image.shape[0]:
    if image[i+1, j]==1 and visited[i+1, j]==0:
      find_start_end(image, i+1, j, visited, segment_ends, l)
  if j+1<image.shape[1]:
    if image[i, j+1]==1 and visited[i, j+1]==0:
      find_start_end(image, i, j+1, visited, segment_ends, l)
  if i-1>=0 and j-1>=0:
    if image[i-1, j-1]==1 and visited[i-1, j-1]==0:
      find_start_end(image, i-1, j-1, visited, segment_ends, l)
  if i+1<image.shape[0] and j-1>=0:
    if image[i+1, j-1]==1 and visited[i+1,j-1]==0:
      find_start_end(image, i+1, j-1, visited, segment_ends, l)
  if i+1<image.shape[0] and j+1<image.shape[1]:
    if image[i+1, j+1]==1 and visited[i+1, j+1]==0:
      find_start_end(image, i+1, j+1, visited, segment_ends, l)
  if i-1>=0 and j+1<image.shape[1]:
    if image[i-1, j+1]==1 and visited[i-1, j+1]==0:
      find_start_end(image, i-1, j+1, visited, segment_ends, l)
  if get_neighbourhood(image, i, j)==1 or get_neighbourhood(image, i, j)==0:
    segment_ends[i, j]=1
  for x in range(segment_ends.shape[0]):
    for y in range(segment_ends.shape[1]):
      if segment_ends[x,y]==1 and (x,y)!=(i, j):
        return (x,y)

## Function for fuzzy convergence

In [27]:
def fuzzy_convergence(image, R):
  all_edges = np.zeros(image.shape)
  convergence = np.zeros(image.shape)
  for i in range(image.shape[0]):
    for j in range(image.shape[1]):
      if image[i,j]==1 and get_neighbourhood(image, i, j)==1 and all_edges[i,j]==0:
        (x1, y1) = (i,j)
        (x2, y2) = get_edge(image, i, j)

        all_edges[x1, x2] = 1
        all_edges[x2, y2] = 1
        if x1==x2:
          slope = 0
          if inbounds(image.shape, (x1-15, y1)):
            x1 = x1-15
          if inbounds(image.shape, (x2+15, y2)):
            x2 = x2+15
        elif y1==y2:
          slope = math.pi/2
          if inbounds(image.shape, (x1, y1-15)):
            y1 = y1-15
          if inbounds(image.shape, (x2, y2+15)):
            y2 = y2+15
        else:
          slope = math.atan((x2-x1)/(y2-y1))
          x1_new = int(x1-slope*15)
          y1_new = int(y1-15/slope)
          if inbounds(image.shape, (x1_new, y1_new)):
            x1 = x1_new
            y1 = y1_new
          x2_new = int(x2+slope*15)
          y2_new = int(y2-15/slope)
          if inbounds(image.shape, (x2_new, y2_new)):
            x2 = x2_new
            y2 = y2_new
        alpha = math.pi/2 - slope
        for theta in range(0, 360):
          theta = theta+math.pi/180
          t = 0
          while t<=1:
            x = int (x1 + R*math.cos(alpha+theta)+t*(x2-x1-2*R*math.cos(theta)*math.cos(alpha)))
            y = int (y1 + R*math.sin(alpha+theta)+t*(y2-y1-2*R*math.cos(theta)*math.sin(alpha)))
            if inbounds(image.shape, (x,y)):
              convergence[x,y]+=1
            t+=0.1
  optic_nerve_index = np.unravel_index(np.argmax(convergence), convergence.shape)
  return optic_nerve_index

## Setting all parameters

In [32]:
# All parameters:
loop_limit = 5000

thresh = 5000
smin = 140
smax = 3000
fringe = 0.18
tree = 252

t1 = 15
t2 = 100
R = 15

## Main program - to find the performance of the algorithm

In [ ]:
errors = []
ground_truth = open("GT_NERVES.txt")
lines = ground_truth.readlines()
for iterator in range(1, 21):
  line = lines[iterator-1]
  name, true_x, true_y = line.split(" ")
  if iterator<10:
    name = "/content/drive/MyDrive/images/im000" + str(iterator) + ".ppm"
  else:
    name = "/content/drive/MyDrive/images/im00" + str(iterator) + ".ppm"
  image = cv2.imread(name)
  image = image[:,:,1]

  conv_image = convolution_segmentation(image)
  img = conv_image.astype(np.int16)

  probe = Probe(thresh, smin, smax, fringe, tree)
  temp = []
  tempimg = img.copy()
  que = probe.init_queue(tempimg)
  num = 0
  vessel = np.zeros_like(img)

  while num < len(que) and num < loop_limit:
      tempvessel = np.zeros_like(img)
      T = img[que[num][1]][que[num][0]]
      if T <= 0:
          num += 1
          continue
      size = 0
      (size, piece, listcd) = probe.paint_fill(img, tempvessel, que[num], T)
      while probe.tests(size, piece, T, vessel, listcd):
          T -= 1
          if T <= 0:
              break
          (size, piece, listcd) = probe.paint_fill(img, tempvessel, que[num], T)
      if size < smax and size > smin or touchpieces(vessel, temp):
          temp.append(tempvessel)
          vessel = probe.label(vessel, tempvessel)
          que = probe.addpoints(que, tempvessel, vessel, listcd)
          que = probe.deletepoint(que, vessel, num)
      else:
          pass
      num += 1

  segmented = np.zeros(img.shape)
  for i in range(img.shape[0]):
    for j in range(img.shape[1]):
      if vessel[i,j]:
        segmented[i,j]= img[i,j]

  thinned = thin(vessel)
  broken = break_image(thinned)

  broken,_ = get_segments(broken, t1, t2)
  index = fuzzy_convergence(broken, R)
  true_x = int(true_x)
  true_y = int(true_y)
  print (iterator, true_x, true_y, index)
  errors.append(math.sqrt((true_x - index[0])**2 + (true_y - index[1])**2))
print ("Mean Error = ", sum(errors)/len(errors))
success = 0
for i in range(len(errors)):
  if errors[i]<=100:
    success+=1
print ("Number of successful predictions = ", success)